In [1]:
# === Bootstrap cell: shared config, paths, JSON I/O, and GitHub REST/GraphQL helpers ===

import os, json, time, re, random, pathlib
from typing import Dict, Any, Optional, Tuple, List
from datetime import datetime, timezone

import requests
import yaml
from dotenv import load_dotenv

# -------------------------------------------------------------------
# Locate repo root reliably (works no matter where you open the .ipynb)
# -------------------------------------------------------------------
def _find_repo_root(start: pathlib.Path) -> pathlib.Path:
    cur = start.resolve()
    for _ in range(8):  # walk up to 8 levels
        if (cur / "config" / "config.yaml").exists():
            return cur
        cur = cur.parent
    raise FileNotFoundError("Couldn't locate repo root (no config/config.yaml found upward).")

REPO_ROOT = _find_repo_root(pathlib.Path.cwd())
CONFIG_PATH = REPO_ROOT / "config" / "config.yaml"

# -------------------------------------------------------------------
# Load config + token
# -------------------------------------------------------------------
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CFG: Dict[str, Any] = yaml.safe_load(f) or {}

load_dotenv(REPO_ROOT / ".env")  # local only; do not commit .env
TOKEN_ENV = (CFG.get("github_token_env") or "GITHUB_TOKEN").strip()
GITHUB_TOKEN = os.getenv(TOKEN_ENV, "").strip()
if not GITHUB_TOKEN:
    raise RuntimeError(f"Missing token in environment variable {TOKEN_ENV}. "
                       f"Create .env from .env.example and set {TOKEN_ENV}=...")

# Repo + output root
REPO = CFG["repo"]  # "owner/name"
OWNER, NAME = REPO.split("/", 1)
OUT_ROOT = pathlib.Path(CFG.get("out_root") or NAME)  # "<name>" if null

# Behavior
OVERWRITE = bool(CFG.get("overwrite", True))
VERBOSE = bool(CFG.get("verbose_logs", True))

# Networking knobs
REQ_TIMEOUT = int(CFG.get("request_timeout_sec", 30))
MAX_RETRIES = int(CFG.get("max_retries", 4))
BACKOFF_BASE_MS = int(CFG.get("backoff_base_ms", 400))
BACKOFF_JITTER_MS = int(CFG.get("backoff_jitter_ms", 250))
RESPECT_RL = bool(CFG.get("respect_rate_limits", True))

# -------------------------------------------------------------------
# Tiny logging helpers
# -------------------------------------------------------------------
def log(msg: str) -> None:
    if VERBOSE:
        print(msg, flush=True)

def warn(msg: str) -> None:
    print(f"⚠️  {msg}", flush=True)

print("Using repo root:", REPO_ROOT)

# -------------------------------------------------------------------
# Canonical output layout helpers
# -------------------------------------------------------------------
def ensure_dir(p: pathlib.Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def repo_root() -> pathlib.Path:
    root = OUT_ROOT
    ensure_dir(root)
    return root

def tags_all_json() -> pathlib.Path:
    d = repo_root() / "tags"
    ensure_dir(d)
    return d / "tags.all.json"

def series_dir(kind: str, series: str) -> pathlib.Path:
    d = repo_root() / kind / series
    ensure_dir(d)
    return d

def pair_stem(base: str, compare: str) -> str:
    return f"{base}...{compare}"

def pair_json(series: str, stem: str, kind: str) -> pathlib.Path:
    # kind ∈ {"compare","commits","pulls","issues"}
    return series_dir(kind, series) / f"{stem}.{kind}.json"

def capsule_json(series: str, stem: str) -> pathlib.Path:
    return series_dir("commits_pr_issue", series) / f"{stem}.tarce_artifacts.json"

# -------------------------------------------------------------------
# JSON I/O
# -------------------------------------------------------------------
def read_json(path: pathlib.Path) -> Optional[Dict[str, Any]]:
    if not path.exists():
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(path: pathlib.Path, payload: Dict[str, Any]) -> None:
    ensure_dir(path.parent)
    if path.exists() and not OVERWRITE:
        log(f"Skip (exists): {path}")
        return
    tmp = path.with_suffix(path.suffix + ".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    tmp.replace(path)
    log(f"✓ Wrote {path}")

# -------------------------------------------------------------------
# GitHub HTTP session + rate-limit aware helpers
# -------------------------------------------------------------------
SESSION = requests.Session()
SESSION.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": f"notebook-pipeline/{NAME}"
})

def _parse_reset_epoch(headers: Dict[str, Any]) -> Optional[int]:
    try:
        return int(headers.get("X-RateLimit-Reset") or headers.get("x-ratelimit-reset"))
    except Exception:
        return None

def _maybe_sleep_for_reset(resp: requests.Response) -> None:
    if not RESPECT_RL:
        return
    remaining = resp.headers.get("X-RateLimit-Remaining") or resp.headers.get("x-ratelimit-remaining")
    if remaining is not None and str(remaining).isdigit() and int(remaining) <= 0:
        reset_epoch = _parse_reset_epoch(resp.headers)
        if reset_epoch:
            now = int(time.time())
            delta = max(0, reset_epoch - now) + 1
            warn(f"Rate limit reached. Sleeping ~{delta}s until reset …")
            time.sleep(delta)

def _backoff_sleep(i: int) -> None:
    base = BACKOFF_BASE_MS / 1000.0
    jitter = random.uniform(0, BACKOFF_JITTER_MS / 1000.0)
    time.sleep((2 ** i) * base + jitter)

def rest_get_json(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Generic GET with backoff + rate-limit handling.
    """
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.get(url, params=params, timeout=REQ_TIMEOUT)
            # Honor 429 Retry-After if present
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            return r.json()
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GET failed after retries: {url} :: {last_err}")

def gh_graphql(query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    GitHub GraphQL POST with backoff + rate-limit handling.
    Endpoint: https://api.github.com/graphql
    """
    url = "https://api.github.com/graphql"
    payload = {"query": query, "variables": variables or {}}
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.post(url, json=payload, timeout=REQ_TIMEOUT)
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"GraphQL HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            data = r.json()
            if "errors" in data:
                last_err = f"GraphQL errors: {data['errors']}"
                _backoff_sleep(i)
                continue
            return data
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GraphQL failed after retries: {last_err}")

def rate_limit_snapshot() -> Dict[str, Any]:
    """
    Returns current REST rate-limit bucket (printed; not written to JSON outputs).
    """
    try:
        info = rest_get_json("https://api.github.com/rate_limit")
        core = info.get("resources", {}).get("core", {})
        remaining = core.get("remaining")
        limit = core.get("limit")
        reset = core.get("reset")
        when = datetime.fromtimestamp(reset, tz=timezone.utc).isoformat() if reset else None
        snap = {"limit": limit, "remaining": remaining, "reset_epoch": reset, "reset_iso": when}
        log(f"Rate limit: {remaining}/{limit}, resets at {when}")
        return snap
    except Exception as e:
        warn(f"Rate limit snapshot failed: {e}")
        return {}

# -------------------------------------------------------------------
# Light helpers for series + ordering
# -------------------------------------------------------------------
def semver_series(tag_name: str) -> str:
    """
    Extract 'vX.Y' series from tags like 'v4.2.3', '4.2.0-rc.1', etc.
    Falls back to 'v0.0' if not parseable.
    """
    m = re.search(r'v?(\d+)\.(\d+)', tag_name or "")
    if not m:
        return "v0.0"
    return f"v{int(m.group(1))}.{int(m.group(2))}"

def sorted_pairs_by_tag_time(pairs: List[Tuple[str, str]], tag_index: Dict[str, Dict[str, Any]]) -> List[Tuple[str, str]]:
    """
    Sort (base, compare) by tag timestamps ascending when both are tags;
    unknown timestamps sort last.
    """
    def ts(tag: str) -> float:
        rec = tag_index.get(tag)
        if rec and rec.get("tag_timestamp"):
            try:
                return datetime.fromisoformat(rec["tag_timestamp"].replace("Z", "+00:00")).timestamp()
            except Exception:
                pass
        return float("inf")
    return sorted(pairs, key=lambda bc: (ts(bc[0]), ts(bc[1])))

log("Bootstrap ready ✓")


Using repo root: /Users/rambodparsi/Desktop/OSS Repository Selection/web_scrapper/commit_pr_issue_analysis
Bootstrap ready ✓


### Config echo

In [2]:
print("Repo:", REPO)
print("Output root:", repo_root())
print("Overwrite:", OVERWRITE, "| Verbose:", VERBOSE)
rate_limit_snapshot();  # prints remaining + reset time


Repo: mastodon/mastodon
Output root: mastodon
Overwrite: True | Verbose: True
Rate limit: 4322/5000, resets at 2025-10-26T05:50:50+00:00


### Fetch & timestamp all tags, then save tags.all.json

In [4]:
from typing import Dict, Any, List, Optional

def list_repo_tags(owner: str, name: str, per_page: int = 100) -> List[Dict[str, Any]]:
    """Paginate /repos/{owner}/{repo}/tags (name + commit.sha)."""
    url = f"https://api.github.com/repos/{owner}/{name}/tags"
    tags = []
    page = 1
    while True:
        data = rest_get_json(url, params={"per_page": per_page, "page": page})
        if not isinstance(data, list) or not data:
            break
        tags.extend(data)
        page += 1
    return tags

def get_ref_for_tag(owner: str, name: str, tag: str) -> Optional[Dict[str, Any]]:
    """
    /repos/{owner}/{repo}/git/refs/tags/{tag}
    Returns a ref object pointing to either an annotated tag object (type 'tag') or directly to a commit (type 'commit').
    """
    url = f"https://api.github.com/repos/{owner}/{name}/git/refs/tags/{tag}"
    try:
        return rest_get_json(url)
    except Exception as e:
        warn(f"ref lookup failed for tag {tag}: {e}")
        return None

def get_git_tag(owner: str, name: str, tag_sha: str) -> Optional[Dict[str, Any]]:
    """If annotated: /repos/{owner}/{repo}/git/tags/{sha} -> has tagger.date."""
    url = f"https://api.github.com/repos/{owner}/{name}/git/tags/{tag_sha}"
    try:
        return rest_get_json(url)
    except Exception as e:
        warn(f"git tag lookup failed for sha {tag_sha}: {e}")
        return None

def get_commit(owner: str, name: str, sha: str) -> Optional[Dict[str, Any]]:
    """Fallback for lightweight tags: /repos/{owner}/{repo}/commits/{sha} -> commit.author/committer.date."""
    url = f"https://api.github.com/repos/{owner}/{name}/commits/{sha}"
    try:
        return rest_get_json(url)
    except Exception as e:
        warn(f"commit lookup failed for sha {sha}: {e}")
        return None

def resolve_tag_timestamp(owner: str, name: str, tag_name: str, fallback_commit_sha: str) -> (Optional[str], str):
    """
    Try to get the most appropriate timestamp for the tag:
      1) If ref->object.type == 'tag' (annotated), use git tag's tagger.date.
      2) Else (lightweight), use commit.committer.date; if missing, use commit.author.date.
    Returns (iso_timestamp, source).
    """
    ref = get_ref_for_tag(owner, name, tag_name)
    if ref:
        obj = ref.get("object") or {}
        obj_type = obj.get("type")
        obj_sha = obj.get("sha")
        if obj_type == "tag" and obj_sha:
            t = get_git_tag(owner, name, obj_sha)
            if t:
                tagger = t.get("tagger") or {}
                dt = tagger.get("date")
                if dt:
                    return dt, "tag"
            # if annotated but no tagger date, fall through to commit fallback below
        elif obj_type == "commit" and obj_sha:
            fallback_commit_sha = obj_sha

    # Fallback to commit timestamp (lightweight tag or missing tagger date)
    c = get_commit(owner, name, fallback_commit_sha)
    if c:
        meta = c.get("commit") or {}
        committer = (meta.get("committer") or {})
        author = (meta.get("author") or {})
        dt = committer.get("date") or author.get("date")
        if dt:
            return dt, "commit"

    return None, "unknown"

# 1) list tags (name + commit.sha)
raw_tags = list_repo_tags(OWNER, NAME)
log(f"Found {len(raw_tags)} tags from /repos/:owner/:repo/tags")

# 2) enrich with timestamps + series
enriched: List[Dict[str, Any]] = []
for t in raw_tags:
    tname = t.get("name")
    csha = (t.get("commit") or {}).get("sha")
    if not tname or not csha:
        continue

    ts, src = resolve_tag_timestamp(OWNER, NAME, tname, csha)
    enriched.append({
        "name": tname,
        "sha": csha,
        "zipball_url": f"https://api.github.com/repos/{OWNER}/{NAME}/zipball/refs/tags/{tname}",
        "tarball_url": f"https://api.github.com/repos/{OWNER}/{NAME}/tarball/refs/tags/{tname}",
        "series": semver_series(tname),
        "tag_timestamp": ts,
        "timestamp_source": src
    })

# 3) sort old→new by tag_timestamp (None at end)
def ts_key(x: Dict[str, Any]) -> float:
    val = x.get("tag_timestamp")
    if not val:
        return float("inf")
    try:
        return datetime.fromisoformat(val.replace("Z","+00:00")).timestamp()
    except Exception:
        return float("inf")

enriched.sort(key=ts_key)

# 4) write <name>/tags/tags.all.json
payload = {
    "repo": REPO,
    "tags": enriched
}
write_json(tags_all_json(), payload)

rate_limit_snapshot()  # print at end for visibility


Found 312 tags from /repos/:owner/:repo/tags
✓ Wrote mastodon/tags/tags.all.json
Rate limit: 3536/5000, resets at 2025-10-26T05:50:50+00:00


{'limit': 5000,
 'remaining': 3536,
 'reset_epoch': 1761457850,
 'reset_iso': '2025-10-26T05:50:50+00:00'}